In [1]:
from itertools import combinations, chain #do not import anything else 
# (you may or may not use combinations -- up to you --
#  see https://docs.python.org/3/library/itertools.html?highlight=combinations#itertools.combinations
#  and https://docs.python.org/3/library/itertools.html?highlight=chain#itertools.chain)

# prints out a set, nicely
# names is an optional list of the names for each of the (integer) items
def settostr(s,names=None):
    if names is None:
        elems = [str(e) for e in s]
    else:
        elems = [names[e] for e in s]
    return "{" + (", ".join(elems)) + "}"

In [2]:
# loads in data from filename, assuming the file format used for this assignment
def loaddata(filename):
    with open(filename) as f:
        nitems = int(f.readline())
        names = [f.readline().strip() for i in range(nitems)]
        nrows = int(f.readline())
        data = [[int(s) for s in f.readline().split()] for i in range(nrows)]
        f.close()
        return (names,data,nitems)        

In [3]:
# def getFreqCount(data, s):
#     # dataSet = set(frozenset(i) for i in data)
#     # print(dataSet)
#     cnt = 0
#     for i in data:
#         flag = True
#         for j in s:
#             if j not in i:
#                 flag = False
#                 break
#         if flag:
#             cnt += 1

#     return cnt

In [4]:
# # Hashing the frequency count of potential freq item sets.
# global FreqCount
# FreqCount = {}

In [5]:
# def getFreqCount(dataSet, s):
#     # s = set(s)
    
    
#     ls = list(s)
    
#     ls.sort()
    
#     key = ''.join(str(x) for x in s)
    
#     if key in FreqCount:
#         return FreqCount[key]
    
#     # print(dataSet)
    
#     # print(dataSet)
#     cnt = 0
#     for i in dataSet:
#         if s <= set(i):
#             cnt += 1
#             # print(set(i))
            
            
# #         flag = True
# #         for j in s:
# #             if j not in i:
# #                 flag = False
# #                 break
# #         if flag:
# #             cnt += 1

#     FreqCount[key] = cnt
    
#     return cnt

In [6]:
# def getFreqCount(data, s):
    
#     dataSet = []
#     for i in data:
#         dataSet.append(set(i))
#     cnt = 0
#     for i in dataSet:
#         if s <= i:
#             cnt += 1
    
#     return cnt

In [7]:
# works the best
def getFreqCount(data, s):

    cnt = 0
    for i in data:
        flag = True
        for j in s:
            if j not in i:
                flag = False
                break
        if flag:
            cnt += 1

    return cnt

In [8]:
def aprioriGen(L, frequentSetSize):
    frequentItemSet = set(frozenset())
    firsTPass = True
    for i in L:
        for j in L:
            if i != j and len(i.union(j)) == frequentSetSize:
                frequentItemSet = frequentItemSet.union({i.union(j)})        
    return frequentItemSet

In [9]:
# returns all the subsets except {}, {all the elements} because we cannot make rule from them
def GenerateSubsets(my_set):
    subsets = []
    for i in range(1, len(my_set)):
        subsets.extend(list(itertools.combinations(my_set, i)))
    return [set(s) for s in subsets]

In [10]:
# learnrules takes
#   numitems (number of total items possible)
#   data (a list of lists.  data[i] is the ith example. data[i][j] is an integer indicating the jth item in the ith itemset)
#   minsupport (the minimum support)
#   minconfidence (the minimum confidence)
# and returns
#   a list or set of rules
#     (each rule is a pair of two sets: the lefthand and righthand sides.  These sets are of integers, referring to items)
def learnrules(numitems,data,minsupport,minconfidence):
    ## YOUR CODE HERE (you may add helper functions as well)
    
    
     # 1 Generating frequent item sets
    
    
    dataLenght = len(data)
    
    #initializing L containing frequent sets of lenght 1
    L = [[c] for c in range(numitems)]
    L = set(frozenset(i) for i in L)
    

    # dataSet = data
    
    # Droping the values in L who dont meet minimum support condition.
    L  = set(i for i in L if getFreqCount(data, i)/ dataLenght >= minsupport)
    
    i = 1
    
    
    # Contains all the frequent item sets.
    RES = set(frozenset())
    
    
    while(len(L) > 0):
        
        i = i + 1
        
        
        # From frequent items sets of size i, generate candidate frequent items sets of size i + 1
        C = aprioriGen(L, i)
        
        L = set(frozenset())
        
        # Adding only those values to  L who meet minimum support condition.
        for s in C:
            if(getFreqCount(data, s)/ dataLenght >= minsupport):
                L = L.union({s})
        
        # adding fequent sets of lenght i to the final result
        RES = RES.union(L)
    
    # print(RES)
    
    # 2 - forming rules
    rules = []
    for Z in RES:
        subsets = GenerateSubsets(Z)
        for X in subsets:
            # Right part of the rule
            Y = (Z - X)
            L = []
            #only adding those rules who follow the minimum confidence condition
            if(getFreqCount(data, Z)/getFreqCount(data, X) >= minconfidence):
                
                rule = [X, set(Y)]
                

                rules.append(rule)
    # print(rules)
    
    return rules

In [11]:
def writerules(rules,data,itemnames):
    ### ADD YOUR CODE HER
    res = []

  # 1 sorting based on confidence and storing the rows in list
    for i in rules:
        X = i[0]
        Y = i[1]
        Z = i[0].union(i[1])
        support = getFreqCount(data, Z)/len(data)
        confidence = getFreqCount(data, Z)/getFreqCount(data, X)
        leftSet = set()
        rightSet = set()
        for j in X:
            leftSet.add(itemnames[j])
        for j in Y:
            rightSet.add(itemnames[j])

        res.append([support,confidence, leftSet, rightSet])
        
    res = sorted(res,key=lambda l:l[1], reverse = True)  
    

    
    #2 printing the results
    
    for i in res:
        s = " "
        k = 0
        size = len(i)
        for j in i:
            if k <= 1:
                s += "{:7.4f}".format(j)
            else:
                s += str(j)
            if k == 2:
                s += "  =>"
            k += 1
            if k <= size:
                s += "  "
        print(s)
                
    ## should print out each rule, *sorted by confidence*, (high to low) one per line
    ## each line should list the support, then the confidence, then the rule
    ## to line up the columns nicely, use
    ##       "{:7.4f}".format(x)
    ## (or equivalent) to print the floating point number in the variable x
    ## use settostr (above) to write out the itemsets  

In [12]:
# # prints the rule set
# def printruleset(datasetfilename,minsupport,minconfidence):
#     (itemnames,data,numitems) = loaddata(datasetfilename)
#     rules = learnrules(numitems,data,minsupport,minconfidence)
#     writerules(rules,data,itemnames)

In [13]:
# prints the rule set
def printruleset(datasetfilename,minsupport,minconfidence):
    (itemnames,data,numitems) = loaddata(datasetfilename)
    # print(data)
    rules = learnrules(numitems,data,minsupport,minconfidence)
    writerules(rules,data,itemnames)

In [17]:
# ## toy dataset example
# printruleset('/usr/local/cs171/toymovies.txt',0.3,0.5)
# ''' output should look like
#  0.5000  1.0000    {Toy Story} => {Star Wars}
#  0.3000  1.0000    {Star Wars, Home Alone} => {Toy Story}
#  0.3000  1.0000    {Home Alone, Toy Story} => {Star Wars}
#  0.5000  0.7143    {Star Wars} => {Toy Story}
#  0.3000  0.6000    {Star Wars, Toy Story} => {Home Alone}
#  0.3000  0.6000    {Toy Story} => {Home Alone}
#  0.3000  0.6000    {Toy Story} => {Star Wars, Home Alone}
#  0.3000  0.5000    {Home Alone} => {Toy Story}
#  0.3000  0.5000    {Home Alone} => {Star Wars, Toy Story}
#  0.3000  0.5000    {Home Alone} => {Star Wars}
# ''';

In [18]:
# %%time
# # the full groceries answer (should take under a minute to run)
# printruleset('/usr/local/cs171/groceries.txt',0.01,0.5)

In [19]:
# %%time
# # a different groceries dataset, looking for rules with less support, but higher confidence (should take under a minute to run)
# printruleset('/usr/local/cs171/groceries2.txt',0.001,0.7)